In [1]:
import numpy as np
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
import pandas as pd
import mlflow
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import set_config
from sklearn.metrics import accuracy_score

C:\Users\hp\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Load the dataset
X,y = load_iris(return_X_y=True,as_frame=True)

X

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                  5.1               3.5                1.4               0.2
1                  4.9               3.0                1.4               0.2
2                  4.7               3.2                1.3               0.2
3                  4.6               3.1                1.5               0.2
4                  5.0               3.6                1.4               0.2
..                 ...               ...                ...               ...
145                6.7               3.0                5.2               2.3
146                6.3               2.5                5.0               1.9
147                6.5               3.0                5.2               2.0
148                6.2               3.4                5.4               2.3
149                5.9               3.0                5.1               1.8

[150 rows x 4 columns]

In [3]:
set_config(transform_output='pandas')

In [4]:
# train test split

random_state = 42
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size,random_state=random_state)

print('The shape of the train data is',X_train.shape)

The shape of the train data is (120, 4)


In [5]:
scaler = StandardScaler()

label_encoder = LabelEncoder()

# fit_transform on the train data
X_train_trans = scaler.fit_transform(X_train)
y_train = label_encoder.fit_transform(y_train)

# transform on the test data
X_test_trans = scaler.transform(X_test)
y_test = label_encoder.transform(y_test)

In [6]:
X_train_trans

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
22           -1.473937          1.203658          -1.562535         -1.312603
15           -0.133071          2.992376          -1.276006         -1.045633
65            1.085898          0.085709           0.385858          0.289218
11           -1.230143          0.756479          -1.218701         -1.312603
42           -1.717731          0.309299          -1.390618         -1.312603
..                 ...               ...                ...               ...
71            0.354517         -0.585060           0.156636          0.155733
106          -1.108246         -1.255829           0.443164          0.689673
14           -0.011174          2.098017          -1.447923         -1.312603
92           -0.011174         -1.032239           0.156636          0.022248
102           1.573486         -0.137880           1.245443          1.223613

[120 rows x 4 columns]

In [7]:
# define the objective function

def objective(params):
    with mlflow.start_run(nested=True) as parent:
        # log the current run parameters
        mlflow.log_params(params)
        # give the parameters as integers
        max_depth = int(params['max_depth'])
        n_estimators = int(params['n_estimators'])
        # train the model
        model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth)
        model.fit(X_train_trans,y_train)
        # get the accuracy score
        y_pred = model.predict(X_test_trans)
        # log the score
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric('accuracy_score',accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK}

In [8]:
# define the search space

space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 10),
    'max_depth': hp.quniform('max_depth', 5, 20, 1),
    'criterion': hp.choice('criterion',['gini','entropy'])

}

In [9]:
# set the tracking uri

mlflow.set_tracking_uri("http://127.0.0.1:8080")

mlflow.set_experiment("HP Tuning Hyperopt")

with mlflow.start_run(run_name='Main Run'):
    # set the fmin function
    trials = Trials()
    
    best = fmin(fn=objective, 
                space=space, 
                algo=tpe.suggest, 
                max_evals=50, 
                trials=trials)

2024/10/30 19:19:03 INFO mlflow.tracking.fluent: Experiment with name 'HP Tuning Hyperopt' does not exist. Creating a new experiment.


  0%|                                                                           | 0/50 [00:00<?, ?trial/s, best loss=?]

2024/10/30 19:19:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run brawny-frog-267 at: http://127.0.0.1:8080/#/experiments/322195731213365606/runs/4b02535a6c4f4faa9b04ec48dfe0eadf.

2024/10/30 19:19:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/322195731213365606.



  2%|█▎                                                             | 1/50 [00:00<00:33,  1.46trial/s, best loss: -1.0]

2024/10/30 19:19:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run unleashed-trout-303 at: http://127.0.0.1:8080/#/experiments/322195731213365606/runs/8c490f21294f4eb2953f038d0f35e4c3.

2024/10/30 19:19:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/322195731213365606.



...

2024/10/30 19:19:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run sassy-perch-120 at: http://127.0.0.1:8080/#/experiments/322195731213365606/runs/e0c9f52db79143288a05e550aa00b1b9.

2024/10/30 19:19:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/322195731213365606.



 98%|████████████████████████████████████████████████████████████▊ | 49/50 [00:46<00:00,  1.02trial/s, best loss: -1.0]

2024/10/30 19:19:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run delicate-shark-256 at: http://127.0.0.1:8080/#/experiments/322195731213365606/runs/5599833d0f9c407baae909e1830f4ffc.

2024/10/30 19:19:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/322195731213365606.



100%|██████████████████████████████████████████████████████████████| 50/50 [00:46<00:00,  1.07trial/s, best loss: -1.0]

2024/10/30 19:19:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run Main Run at: http://127.0.0.1:8080/#/experiments/322195731213365606/runs/c408180d16a7464288b199be7dd24e60.
2024/10/30 19:19:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/322195731213365606.


In [10]:
best

{'criterion': 0, 'max_depth': 9.0, 'n_estimators': 140.0}